In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 15 14:49:58 2022

@author:Unknown

Functional autoencoder implementation
"""

# Import modules
import inspect
import random
import torch
import torch.nn.init as init
import torch.nn.functional as F
import torch.nn as nn

from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from numpy import *
import seaborn as sns
import matplotlib
#matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
# import skfda as fda
# from skfda import representation as representation
# from skfda.exploratory.visualization import FPCAPlot
# # from skfda.exploratory.visualization import FPCAPlot
# # from skfda.preprocessing.dim_reduction import FPCA
# # from skfda.representation.basis import BSpline, Fourier, Monomial
import scipy
from scipy.interpolate import BSpline
import os
import ignite
from tqdm import tqdm
import sklearn
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import random
from random import seed
from scipy import stats
import statistics
from statistics import stdev

import skfda
from skfda import FDataGrid as fd
from skfda.representation.basis import BSpline as B


In [3]:
import pandas as pd
import os
os.chdir("C:/Users/Utilisateur/Documents/Master_SSD/STAGE_RECHERCHE/Functionnal Classifier")
# Lire le fichier CSV
sofa = pd.read_csv('sofa.csv')
T=20
# Attribuer des noms de colonnes personnalisés


# Afficher les noms de colonnes
print(sofa.columns)
grid_points=np.arange(T+1)[1:]
sofa.values.shape


Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       ...
       'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 'V171', 'V172', 'V173',
       'V174'],
      dtype='object', length=174)


(520, 174)

In [4]:
Y_X=torch.from_numpy(sofa.values).float().cuda()

y,x=Y_X[:,0].unsqueeze(1).unsqueeze(2).long(),Y_X[:,1:T+1].reshape(520,1,T)
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(x,y,shuffle=True)
x_train.shape,y_train.shape


(torch.Size([390, 1, 20]), torch.Size([390, 1, 1]))

In [5]:

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.07)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.07)
        torch.nn.init.constant_(m.bias.data, 0.0)


In [6]:
n_epochs=25  
basis=B(knots=linspace(1,T,6),order=3)
n_conv_out=64
granulation=2000
n_conv_in=128
n_conv_in2=128
n_conv_in3=64
n_Flat_out=256
stride_1=6
stride_2=2
stride_3=1
stride_pool_1=2
stride_pool_2=2
stride_pool_3=1
kernel_size_1=200
kernel_size_2=50
kernel_size_3=20
kernel_size_pool_1=6
kernel_size_pool_2=4
kernel_size_pool_3=2
dilation_1=1
dilation_2=1
dilation_3=1
dilation_pool_1=1
dilation_pool_2=1
dilation_pool_3=1
padding_1=2
padding_2=2
padding_3=2
padding_pool_1=1
padding_pool_2=1
padding_pool_3=1
opt="Adam"
lr_adam=0.00089
lr_SGD=0.001
loss_function=nn.CrossEntropyLoss()
batch_size=96

In [7]:
class HyperParameters:
    def __init__(self,batch_size=96, n_epochs=25, basis=B(knots=linspace(1, T, 6), order=3), granulation=2000,
                 n_conv_in=32, n_conv_in2=512, n_conv_in3=256,n_conv_out=64, n_Flat_out=256,
                 stride_1=1, stride_2=1, stride_3=1,
                 stride_pool_1=2, stride_pool_2=2, stride_pool_3=1,
                 kernel_size_1=7, kernel_size_2=4, kernel_size_3=3,
                 kernel_size_pool_1=3, kernel_size_pool_2=3, kernel_size_pool_3=2,
                 dilation_1=1, dilation_2=1, dilation_3=1,
                 dilation_pool_1=1, dilation_pool_2=1, dilation_pool_3=1,
                 padding_1=2, padding_2=2, padding_3=2,
                 padding_pool_1=0, padding_pool_2=0, padding_pool_3=0,
                 opt="Adam", lr=0.00089, loss=nn.CrossEntropyLoss(),activation=nn.Identity()):
        self.n_epochs = n_epochs
        self.batch_size=batch_size
        self.activation=activation
        self.n_conv_out=n_conv_out
        self.basis = basis
        self.granulation = granulation
        self.n_conv_in = n_conv_in
        self.n_conv_in2 = n_conv_in2
        self.n_conv_in3 = n_conv_in3
        self.n_Flat_out = n_Flat_out
        self.stride_1 = stride_1
        self.stride_2 = stride_2
        self.stride_3 = stride_3
        self.stride_pool_1 = stride_pool_1
        self.stride_pool_2 = stride_pool_2
        self.stride_pool_3 = stride_pool_3
        self.kernel_size_1 = kernel_size_1
        self.kernel_size_2 = kernel_size_2
        self.kernel_size_3 = kernel_size_3
        self.kernel_size_pool_1 = kernel_size_pool_1
        self.kernel_size_pool_2 = kernel_size_pool_2
        self.kernel_size_pool_3 = kernel_size_pool_3
        self.dilation_1 = dilation_1
        self.dilation_2 = dilation_2
        self.dilation_3 = dilation_3
        self.dilation_pool_1 = dilation_pool_1
        self.dilation_pool_2 = dilation_pool_2
        self.dilation_pool_3 = dilation_pool_3
        self.padding_1 = padding_1
        self.padding_2 = padding_2
        self.padding_3 = padding_3
        self.padding_pool_1 = padding_pool_1
        self.padding_pool_2 = padding_pool_2
        self.padding_pool_3 = padding_pool_3
        self.opt = opt
        self.lr = lr
        self.loss = loss


In [8]:
arguments_defaults = {
    "n_epochs": 25,
    "basis": B(knots=linspace(1,T,6),order=3),
    "granulation": 2000,
    "n_conv_in": 128,
    "n_conv_in2": 128,
    "n_conv_in3": 64,
    "n_Flat_out": 256,
    "stride_1": 6,
    "stride_2": 2,
    "stride_3": 1,
    "stride_pool_1": 2,
    "stride_pool_2": 2,
    "stride_pool_3": 1,
    "kernel_size_1": 200,
    "kernel_size_2": 50,
    "kernel_size_3": 20,
    "kernel_size_pool_1": 6,
    "kernel_size_pool_2": 4,
    "kernel_size_pool_3": 2,
    "dilation_1": 1,
    "dilation_2": 1,
    "dilation_3": 1,
    "dilation_pool_1": 1,
    "dilation_pool_2": 1,
    "dilation_pool_3": 1,
    "padding_1": 2,
    "padding_2": 2,
    "padding_3": 2,
    "padding_pool_1": 1,
    "padding_pool_2": 1,
    "padding_pool_3": 1,
    "opt": "Adam",
    "lr": 0.00089,
    "loss": nn.CrossEntropyLoss()
}


In [9]:
Defaults_parameters=HyperParameters()


In [10]:

class FCNN(nn.Module):
    def __init__(self, hyperparams):
        super(FCNN, self).__init__()

        basis = hyperparams.basis
        granulation = hyperparams.granulation
        n_conv_in = hyperparams.n_conv_in
        n_conv_in2 = hyperparams.n_conv_in2
        n_conv_in3 = hyperparams.n_conv_in3
        n_Flat_out = hyperparams.n_Flat_out
        stride_1 = hyperparams.stride_1
        stride_2 = hyperparams.stride_2
        stride_3 = hyperparams.stride_3
        stride_pool_1 = hyperparams.stride_pool_1
        stride_pool_2 = hyperparams.stride_pool_2
        stride_pool_3 = hyperparams.stride_pool_3
        kernel_size_1 = hyperparams.kernel_size_1
        kernel_size_2 = hyperparams.kernel_size_2
        kernel_size_3 = hyperparams.kernel_size_3
        kernel_size_pool_1 = hyperparams.kernel_size_pool_1
        kernel_size_pool_2 = hyperparams.kernel_size_pool_2
        kernel_size_pool_3 = hyperparams.kernel_size_pool_3
        dilation_1 = hyperparams.dilation_1
        dilation_2 = hyperparams.dilation_2
        dilation_3 = hyperparams.dilation_3
        dilation_pool_1 = hyperparams.dilation_pool_1
        dilation_pool_2 = hyperparams.dilation_pool_2
        dilation_pool_3 = hyperparams.dilation_pool_3
            
        padding_1 = hyperparams.padding_1
        padding_2 = hyperparams.padding_2
        padding_3 = hyperparams.padding_3
        padding_pool_1 = hyperparams.padding_pool_1
        padding_pool_2 = hyperparams.padding_pool_2
        padding_pool_3 = hyperparams.padding_pool_3

        # Reste du code pour l'initialisation de la classe model

        self.Relu=nn.ReLU()

        self.convlayer1=nn.Sequential(
            nn.Conv1d(1,n_conv_in,kernel_size=kernel_size_1,stride=stride_1,padding=padding_1,dilation=dilation_1),
            nn.BatchNorm1d(n_conv_in),
            nn.LeakyReLU(0.18),
            
            nn.MaxPool1d(kernel_size=kernel_size_pool_1,stride=stride_pool_1,padding=padding_pool_1,dilation=dilation_pool_1),
            nn.BatchNorm1d(n_conv_in),
            nn.LeakyReLU(0.18),
        )
        
        self.convlayer2=nn.Sequential(
            nn.Conv1d(n_conv_in,n_conv_in2,kernel_size=kernel_size_2,stride=stride_2,padding=padding_2,dilation=dilation_2),
            nn.BatchNorm1d(n_conv_in2),
            nn.LeakyReLU(0.18),
            
            nn.MaxPool1d(kernel_size=kernel_size_pool_2,stride=stride_pool_2,padding=padding_pool_2,dilation=dilation_pool_2),
            nn.BatchNorm1d(n_conv_in2),
            nn.LeakyReLU(0.18),
        )
        
        self.convlayer3=nn.Sequential(

            nn.Conv1d(n_conv_in2,n_conv_in3,kernel_size=kernel_size_3,stride=stride_3,padding=padding_3,dilation=dilation_3),
            nn.BatchNorm1d(n_conv_in3),
            nn.LeakyReLU(0.18),
            
            nn.MaxPool1d(kernel_size=kernel_size_pool_3,stride=stride_pool_3,padding=padding_pool_3,dilation=dilation_pool_3),
            nn.BatchNorm1d(n_conv_in3),
            nn.LeakyReLU(0.18),
        )

        self.fc_block=nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_conv_out*n_conv_in3,n_Flat_out),
            nn.BatchNorm1d(n_Flat_out),
            nn.LeakyReLU(0.18),
            
            nn.Linear(n_Flat_out,2),
            
        )
        self.activation=nn.Softmax()
        self.n_conv_out=n_conv_out
        self.n_conv_in=hyperparams.n_conv_in
    def forward(self,x):
       
        Conv_out=self.convlayer1(x)
        Conv_out2=self.convlayer2(Conv_out)
        Conv_out3=self.convlayer3(Conv_out2)
        Lin_out=self.fc_block(Conv_out3)
        return Lin_out.float().unsqueeze_(2).unsqueeze_(3)


In [11]:
def Compile_class(hyperparams):
    basis = hyperparams.basis
    granulation = hyperparams.granulation
    n_conv_in = hyperparams.n_conv_in
    n_conv_in2 = hyperparams.n_conv_in2
    n_conv_in3 = hyperparams.n_conv_in3
    n_Flat_out = hyperparams.n_Flat_out
    stride_1 = hyperparams.stride_1
    stride_2 = hyperparams.stride_2
    stride_3 = hyperparams.stride_3
    stride_pool_1 = hyperparams.stride_pool_1
    stride_pool_2 = hyperparams.stride_pool_2
    stride_pool_3 = hyperparams.stride_pool_3
    kernel_size_1 = hyperparams.kernel_size_1
    kernel_size_2 = hyperparams.kernel_size_2
    kernel_size_3 = hyperparams.kernel_size_3
    kernel_size_pool_1 = hyperparams.kernel_size_pool_1
    kernel_size_pool_2 = hyperparams.kernel_size_pool_2
    kernel_size_pool_3 = hyperparams.kernel_size_pool_3
    dilation_1 = hyperparams.dilation_1
    dilation_2 = hyperparams.dilation_2
    dilation_3 = hyperparams.dilation_3
    dilation_pool_1 = hyperparams.dilation_pool_1
    dilation_pool_2 = hyperparams.dilation_pool_2
    dilation_pool_3 = hyperparams.dilation_pool_3
    padding_1 = hyperparams.padding_1
    padding_2 = hyperparams.padding_2
    padding_3 = hyperparams.padding_3
    padding_pool_1 = hyperparams.padding_pool_1
    padding_pool_2 = hyperparams.padding_pool_2
    padding_pool_3 = hyperparams.padding_pool_3

    # Reste du code de la fonction

    CNN=FCNN(hyperparams=hyperparams).cuda()
    Conv_out=CNN.convlayer1(x_train[:1,:,:])
    Conv_out2=CNN.convlayer2(Conv_out)
    Conv_out3=CNN.convlayer3(Conv_out2)
    n_conv_out=Conv_out3.shape[2]
    hyperparams.n_conv_out=n_conv_out
    class Bsp_classifier(nn.Module): 
        def __init__(self, hyperparams):
            super(Bsp_classifier, self).__init__()

            basis = hyperparams.basis
            granulation = hyperparams.granulation
            n_conv_in = hyperparams.n_conv_in
            n_conv_in2 = hyperparams.n_conv_in2
            n_conv_in3 = hyperparams.n_conv_in3
            n_conv_out=hyperparams.n_conv_out
            n_Flat_out = hyperparams.n_Flat_out
            stride_1 = hyperparams.stride_1
            stride_2 = hyperparams.stride_2
            stride_3 = hyperparams.stride_3
            stride_pool_1 = hyperparams.stride_pool_1
            stride_pool_2 = hyperparams.stride_pool_2
            stride_pool_3 = hyperparams.stride_pool_3
            kernel_size_1 = hyperparams.kernel_size_1
            kernel_size_2 = hyperparams.kernel_size_2
            kernel_size_3 = hyperparams.kernel_size_3
            kernel_size_pool_1 = hyperparams.kernel_size_pool_1
            kernel_size_pool_2 = hyperparams.kernel_size_pool_2
            kernel_size_pool_3 = hyperparams.kernel_size_pool_3
            dilation_1 = hyperparams.dilation_1
            dilation_2 = hyperparams.dilation_2
            dilation_3 = hyperparams.dilation_3
            dilation_pool_1 = hyperparams.dilation_pool_1
            dilation_pool_2 = hyperparams.dilation_pool_2
            dilation_pool_3 = hyperparams.dilation_pool_3
                    
            padding_1 = hyperparams.padding_1
            padding_2 = hyperparams.padding_2
            padding_3 = hyperparams.padding_3
            padding_pool_1 = hyperparams.padding_pool_1
            padding_pool_2 = hyperparams.padding_pool_2
            padding_pool_3 = hyperparams.padding_pool_3


            self.Relu=nn.ReLU()
            
            self.convlayer1=nn.Sequential(
                nn.Conv1d(1,n_conv_in,kernel_size=kernel_size_1,stride=stride_1,padding=padding_1,dilation=dilation_1),
                nn.BatchNorm1d(n_conv_in),
                nn.LeakyReLU(0.18),
                hyperparams.activation,
                nn.MaxPool1d(kernel_size=kernel_size_pool_1,stride=stride_pool_1,padding=padding_pool_1,dilation=dilation_pool_1),
                nn.BatchNorm1d(n_conv_in),
                nn.LeakyReLU(0.18),
                hyperparams.activation,
            )
            
            self.convlayer2=nn.Sequential(
                nn.Conv1d(n_conv_in,n_conv_in2,kernel_size=kernel_size_2,stride=stride_2,padding=padding_2,dilation=dilation_2),
                nn.BatchNorm1d(n_conv_in2),
                nn.LeakyReLU(0.18),
                hyperparams.activation,
                nn.MaxPool1d(kernel_size=kernel_size_pool_2,stride=stride_pool_2,padding=padding_pool_2,dilation=dilation_pool_2),
                nn.BatchNorm1d(n_conv_in2),
                nn.LeakyReLU(0.18),
                hyperparams.activation,
            )
            
            self.convlayer3=nn.Sequential(

                nn.Conv1d(n_conv_in2,n_conv_in3,kernel_size=kernel_size_3,stride=stride_3,padding=padding_3,dilation=dilation_3),
                nn.BatchNorm1d(n_conv_in3),
                nn.LeakyReLU(0.18),
                hyperparams.activation,
                nn.MaxPool1d(kernel_size=kernel_size_pool_3,stride=stride_pool_3,padding=padding_pool_3,dilation=dilation_pool_3),
                nn.BatchNorm1d(n_conv_in3),
                nn.LeakyReLU(0.18),
            )

            self.fc_block=nn.Sequential(
                nn.Flatten(),
                nn.Linear(n_conv_out*n_conv_in3,n_Flat_out),
                nn.BatchNorm1d(n_Flat_out),
                nn.LeakyReLU(0.18),
                hyperparams.activation,
                nn.Linear(n_Flat_out,2),
                
            )
            self.basis=basis
            self.n_conv_out=n_conv_out
       
        def forward(self,x):
            
            
            Conv_out=self.convlayer1(x)
            Conv_out2=self.convlayer2(Conv_out)
            Conv_out3=self.convlayer3(Conv_out2)
            Lin_out=self.fc_block(Conv_out3)
            return Lin_out.float().unsqueeze_(2).unsqueeze_(3)
    module=Bsp_classifier(hyperparams=hyperparams)
    return module

In [12]:
def Compile_train(module, lr, opt="Adam", loss=nn.CrossEntropyLoss(), batch_size=batch_size):
    betas = [0.5, 0.999]
    if opt == "Adam":
        optimizer = optim.Adam(module.parameters(), lr=lr, betas=betas)
    else:
        optimizer = optim.SGD(module.parameters(), lr=lr)
    def train(n_epochs, module, optimizer, loss, batch_size):
        for epoch in range(n_epochs):
            train_loss = torch.tensor(0).cuda().long()
            
            # Mélanger les données d'entraînement
            indices = list(range(len(x_train)))
            random.shuffle(indices)
            
            batch_index = 0  # Indice de batch
            
            for i in range(int(len(x_train) / batch_size)):
                # Obtenir les indices des données mélangées
                batch_indices = indices[batch_index:batch_index+batch_size]
                
                functions_train = x_train[batch_indices,:,:]
                labels_train = y_train[batch_indices]
                
                optimizer.zero_grad()
                output = module(functions_train)
                loss_value = loss(input=output, target=labels_train)
                
                loss_value.backward()
                optimizer.step()
                train_loss += loss_value.long()
                
                batch_index += batch_size  # Passer au prochain batch
                
            return train_loss, loss_value
    
    return lambda n_epochs: train(n_epochs, module, optimizer, loss, batch_size)


In [13]:
def Hyperparameter_Test(hyperparameters):

    x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(x,y,shuffle=True)    
    ##Compilation de la classe 
    Model=Compile_class(hyperparams=hyperparameters).cuda().apply(weights_init_normal)
    
    
    mean_accuracy=torch.tensor([0])
    train_fn = Compile_train(module=Model, lr=lr_adam, opt="Adam", loss=loss_function, batch_size=batch_size)
    for i in tqdm(range(hyperparameters.n_epochs)):
            train_fn(n_epochs=1)
            
            accuracy=((torch.sum(torch.argmax(Model(x_test),dim=1)==y_test)/x_test.shape[0])*100)
            mean_accuracy=torch.cat([mean_accuracy,accuracy.cpu().unsqueeze(0)],dim=0)
    print("Précision moyenne =",((torch.mean(mean_accuracy[1:].float()))).detach().cpu().numpy(),"%")  
    print("Précision max=",((torch.max(mean_accuracy[1:].float()))).detach().cpu().numpy(),"%")  
    print("Variance des précisions =",((torch.var(mean_accuracy[1:].float()))).detach().cpu().numpy())  
    # print(mean_accuracy.unsqueeze(1)[1:]) 
    n_epochs_max_acc=torch.argmax(mean_accuracy)    
    
    
    grid_test=fd(x_test[:,0,:].cpu(),grid_points=grid_points).to_basis(basis)
    grid_convlayer1=fd(Model.convlayer1(x_test)[0,:,:].detach().cpu(),grid_points=linspace(1,T,Model.convlayer1(x_test).shape[2]))
    grid_convlayer2=fd(Model.convlayer2(Model.convlayer1(x_test))[:,0,:].detach().cpu(),grid_points=linspace(1,T,Model.convlayer2(Model.convlayer1(x_test)).shape[2]))
    grid_convlayer3=fd(Model.convlayer3(Model.convlayer2(Model.convlayer1(x_test)))[:,0,:].detach().cpu(),grid_points=linspace(1,T,Model.convlayer3(Model.convlayer2(Model.convlayer1(x_test))).shape[2]))
        
    import gc
    torch.cuda.empty_cache()
    gc.collect()
    return Model,grid_convlayer1,grid_convlayer2,grid_convlayer3, mean_accuracy[1:],n_epochs_max_acc



In [14]:

Model,grid_convlayer1,grid_convlayer2,grid_convlayer3, Acc,n_best=Hyperparameter_Test(hyperparameters=hyperparameters_opt)

NameError: name 'hyperparameters_opt' is not defined

In [ ]:
Hyperparameter_Test(hyperparameters=HyperParameters(n_epochs=50,kernel_size_1=7,n_conv_in=32,n_conv_in2=512,n_conv_in3=256,n_Flat_out=256,stride_1=2))

In [ ]:

grid_convlayer1.plot()
plt.show()

In [56]:
def Hyper_parameter_GridSearch(hyperparams,parameter, grid):
    Total_accuracy_max = torch.tensor([0])
    Optimum_parameter = grid[0]

    
    
    # Obtenir l'attribut correspondant au paramètre spécifié
    attribute = getattr(hyperparams, parameter)
    
    for value in grid:
        # Modifier la valeur de l'attribut de la classe HyperParameters
        setattr(hyperparams, parameter, value)
        print(parameter,"=",value)
        # Utiliser l'instance de HyperParameters pour effectuer les tests
        _, _, _, _, accuracy,n_best = Hyperparameter_Test(hyperparameters=hyperparams)
        Total_accuracy_max = torch.cat([Total_accuracy_max, torch.tensor([torch.max(accuracy)])])

    Optimum_parameter = grid[torch.argmax(Total_accuracy_max[1:])]
    return Optimum_parameter, torch.max(Total_accuracy_max[1:]),n_best


In [57]:
hyperparameters_opt=HyperParameters(n_epochs=50,n_conv_in=32,n_conv_in2=512,n_conv_in3=256,n_Flat_out=256,kernel_size_1=5,)

Optimum_parameter, Acc_max,n_best=Hyper_parameter_GridSearch(hyperparameters_opt,parameter="kernel_size_2",grid=(2,3,4,5,6,7))

kernel_size_2 = 2


100%|██████████| 50/50 [00:01<00:00, 32.93it/s]


Précision moyenne = 94.261536 %
Précision max= 96.15385 %
Variance des précisions = 4.5697374
kernel_size_2 = 3


100%|██████████| 50/50 [00:01<00:00, 37.24it/s]


Précision moyenne = 94.43077 %
Précision max= 96.15385 %
Variance des précisions = 7.8145156
kernel_size_2 = 4


...

Précision moyenne = 95.53846 %
Précision max= 97.69231 %
Variance des précisions = 3.791818
kernel_size_2 = 6


100%|██████████| 50/50 [00:01<00:00, 40.51it/s]


Précision moyenne = 95.40001 %
Précision max= 99.230774 %
Variance des précisions = 10.397059
kernel_size_2 = 7


100%|██████████| 50/50 [00:01<00:00, 40.22it/s]

Précision moyenne = 94.07693 %
Précision max= 96.15385 %
Variance des précisions = 8.700637


In [58]:
hyperparameters_opt.n_conv_in=256
hyperparameters_opt.n_epochs=100
hyperparameters_opt.n_Flat_out=256
hyperparameters_opt.activation=nn.Identity()
hyperparameters_opt.n_conv_in2=64
hyperparameters_opt.kernel_size_3=3
hyperparameters_opt.kernel_size_pool_1

Hyperparameter_Test(hyperparameters_opt)

100%|██████████| 100/100 [00:02<00:00, 37.81it/s]


Précision moyenne = 92.2 %
Précision max= 95.38461 %
Variance des précisions = 10.914105


(Bsp_classifier(
   (Relu): ReLU()
   (convlayer1): Sequential(
     (0): Conv1d(1, 256, kernel_size=(5,), stride=(1,), padding=(2,))
     (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): LeakyReLU(negative_slope=0.18)
     (3): Identity()
     (4): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
     (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (6): LeakyReLU(negative_slope=0.18)
     (7): Identity()
   )
   (convlayer2): Sequential(
     (0): Conv1d(256, 64, kernel_size=(7,), stride=(1,), padding=(2,))
     (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): LeakyReLU(negative_slope=0.18)
     (3): Identity()
     (4): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
     (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (6): LeakyReLU(negative_slope=0.18)
  

In [59]:
n_conv_in_opt,Max_acc,n_bes=Hyper_parameter_GridSearch(hyperparameters_opt,grid = (32,64,128,256),parameter="n_conv_in")

n_conv_in = 32


100%|██████████| 100/100 [00:02<00:00, 38.41it/s]


Précision moyenne = 96.57693 %
Précision max= 98.46154 %
Variance des précisions = 8.073338
n_conv_in = 64


100%|██████████| 100/100 [00:02<00:00, 34.76it/s]


Précision moyenne = 95.36923 %
Précision max= 98.46154 %
Variance des précisions = 4.362923
n_conv_in = 128


100%|██████████| 100/100 [00:02<00:00, 34.88it/s]


Précision moyenne = 93.33847 %
Précision max= 95.38461 %
Variance des précisions = 4.8917527
n_conv_in = 256


 72%|███████▏  | 72/100 [00:02<00:00, 34.89it/s]


KeyboardInterrupt: 

In [25]:
n_conv_in_opt,Max_acc

(32, tensor(97.6923))

In [ ]:
# grid = (32, 64, 128, 256, 512)
# results = []

# for n_conv_in in grid:
#     for n_conv_in2 in grid:
#         for n_conv_in3 in grid:
#             arguments_defaults["n_conv_in"] = n_conv_in
#             arguments_defaults["n_conv_in2"] = n_conv_in2
#             arguments_defaults["n_conv_in3"] = n_conv_in3

#             # Appel de votre fonction avec les paramètres spécifiés
#             _,_,_,_,result = Hyperparameter_Test(**arguments_defaults)

#             # Ajout du résultat à la liste des résultats
#             results.append(result)

# print(results)


In [ ]:

# Hyper_parameter_GridSearch(parameter="n_conv_in3",grid=(32,64,128,256,512),n_epochs=25)

In [ ]:
grids = {
    "n_epochs": 25,
    "basis": B(knots=linspace(1,T,6),order=3),
    "granulation": 2000,
    "n_conv_in": (32,64, 128,256,512),
    "n_conv_in2": (32,64, 128,256,512),
    "n_conv_in3": (32,64, 128,256,512),
    "n_Flat_out": (32,64, 128,256,512),
    "stride_1": np.arange(2)[1:],
    "stride_2": np.arange(2)[1:],
    "stride_3": 1,
    "stride_pool_1": 1,
    "stride_pool_2": 1,
    "stride_pool_3": 1,
    "kernel_size_1": (3,4,5),
    "kernel_size_2": (3,4,5,),
    "kernel_size_3": (3,4,5,),
    "kernel_size_pool_1": np.arange(3)[1:],
    "kernel_size_pool_2": np.arange(2)[1:],
    "kernel_size_pool_3": (1,2),
    "dilation_1": 1,
    "dilation_2": 1,
    "dilation_3": 1,
    "dilation_pool_1": 1,
    "dilation_pool_2": 1,
    "dilation_pool_3": 1,
    "loss": nn.CrossEntropyLoss()
}


In [ ]:
param_list=["stride_1","stride_2","kernel_size_1","kernel_size_3","kernel_size_pool_1","kernel_size_pool_2","kernel_size_3","n_conv_in","n_conv_in2","n_conv_in3","n_Flat_out"]

In [ ]:
def Hyperparameter_Search(hyperparams,grids, parameters):
    best_parameters = {}
    best_accuracy = 0.0

    # Boucle sur les paramètres
    for param in parameters:
        # Boucle sur les grilles pour chaque paramètre
        print(param,"=",grids[param])
            # Appeler la fonction de Grid Search avec les paramètres spécifiés
        optimum_parameter, total_accuracy,n_best = Hyper_parameter_GridSearch(hyperparams,parameter=param,grid=grids[param])

        # Mettre à jour le meilleur résultat si nécessaire
        if total_accuracy > best_accuracy:
            best_parameters = hyperparams
            best_accuracy = total_accuracy
            hyperparams.param=optimum_parameter
    return best_parameters, best_accuracy,n_best


In [ ]:
best_para,best_acc,n_best=Hyperparameter_Search(hyperparams=HyperParameters(),grids=grids,parameters=param_list)

stride_1 = [1]


100%|██████████| 25/25 [00:00<00:00, 29.21it/s]


Précision moyenne = 91.93845 %
Précision max= 95.38461 %
Variance des précisions = 10.360944
stride_2 = [1]


100%|██████████| 25/25 [00:00<00:00, 37.75it/s]


Précision moyenne = 92.307686 %
Précision max= 95.38461 %
Variance des précisions = 13.510844
kernel_size_1 = (3, 4, 5)


...

Précision moyenne = 94.15384 %
Précision max= 96.92308 %
Variance des précisions = 5.7199225


100%|██████████| 25/25 [00:01<00:00, 22.70it/s]


Précision moyenne = 94.184616 %
Précision max= 98.46154 %
Variance des précisions = 13.317554


100%|██████████| 25/25 [00:01<00:00, 21.45it/s]

Précision moyenne = 90.830765 %
Précision max= 94.61538 %
Variance des précisions = 5.568049


In [67]:
best_para.stride_2,best_acc,n_best

(1, tensor(99.2308), tensor(11))

In [43]:
hyperparameters_opt.kernel_size_pool_3

2

In [ ]:
Hyperparameter_Test(hyperparameters_opt)

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()